# Download GEE Imagery

## Step-by-step tutorial

### Import libraries

In [61]:
import ee
import math
import geemap
import sys
import pandas as pd
from multiprocessing import Process, cpu_count 
from geetools import cloud_mask

In [62]:
Map = geemap.Map()
kazbegi = ee.FeatureCollection("users/andihanzl/kazbegi")
points = ee.FeatureCollection("users/andihanzl/points")
Map.centerObject(kazbegi, 10)
Map

Map(center=[42.6252601637589, 44.544784542321295], controls=(WidgetControl(options=['position'], widget=HBox(c…

In [63]:
snow = ee.ImageCollection('MODIS/006/MOD10A1') \
            .select(['NDSI_Snow_Cover'],['Snow']) \
            .filterBounds(kazbegi) \
            .filter(ee.Filter.date('2000-01-01', '2001-01-01')) \
            .sort('system:time_start')

res = snow.first().projection()

snow = snow.map(lambda image: image.reproject(res).clip(kazbegi))

time = ee.ImageCollection('MODIS/006/MYD11A1') \
        .filter(ee.Filter.date('2000-01-01', '2022-12-31')) \
        .select(['Day_view_time']) \
        .sort('system:time_start')
time = time.map(lambda image: image.resample('bicubic').reproject(res).clip(kazbegi).rename('time'))
time = time.map(lambda image: image.expression('time / 10', {'time': image}) \
              .copyProperties(image, ['system:time_start']))

time = time.sort("system:time_start")

count = time.size()
print('Count: ', count.getInfo())

Count:  6850


In [64]:
def getmean(img):
    return img.reduceRegions(collection = kazbegi,
                            reducer = ee.Reducer.mean(),
                            scale = 500)

time = time.map(lambda image: getmean(image))
featureCollection = time.flatten()

task_config = {
    'folder':'bla',
    'fileFormat':'CSV'
}

task = ee.batch.Export.table.toDrive(featureCollection, 'time_lst_aqua_total_new', **task_config)
task.start()